# Python Flight Mechanics Engine 

This installs `pyfme` to be run in this online notebook

In [ ]:
!pip install git+https://github.com/AeroPython/PyFME.git

## Aircraft 

In order to perform a simulation, the first thing we need is an aircraft:

In [1]:
from pyfme.aircrafts import Cessna172

ImportError: No module named 'pyfme'

In [ ]:
aircraft = Cessna172()

Aircraft will provide the simulator the forces, moments and inertial properties in order to perform the integration of the dynamic system equations:

In [ ]:
print(f"Aircraft mass: {aircraft.mass} kg")
print(f"Aircraft inertia tensor: \n {aircraft.inertia} kg/m²")

In [ ]:
print(f"forces: {aircraft.total_forces} N")
print(f"moments: {aircraft.total_moments} N·m")

For the aircraft, in order to calculate its forces and moments it is necessary to set the controls values within the limits: 

In [ ]:
print(aircraft.controls)

In [ ]:
print(aircraft.control_limits)

but also to provide and environment (ie. atmosphere, winds, gravity) and the aircraft state, which will also determine the aerodynamic contribution.

## Environment 

In [ ]:
from pyfme.environment.atmosphere import ISA1976
from pyfme.environment.wind import NoWind
from pyfme.environment.gravity import VerticalConstant

In [ ]:
atmosphere = ISA1976()
gravity = VerticalConstant()
wind = NoWind()

The atmosphere, wind and gravity model make up the environment:

In [ ]:
from pyfme.environment import Environment

In [ ]:
environment = Environment(atmosphere, gravity, wind)

The environment has an update method which given the state (ie. position, altitude...) updates the environment variables (ie. density, wind magnitude, gravity force...)

In [ ]:
help(environment.update)

## State 

Even if the state can be set manually by giving the position, attitude, velocity, angular velocities... Most of the times, the user will want to trim the aircraft in a stationary condition. The aircraft controls to flight in that condition will be also provided by the trimmer.

In [ ]:
from pyfme.utils.trimmer import steady_state_trim

In [ ]:
help(steady_state_trim)

In [ ]:
from pyfme.models.state.position import EarthPosition

In [ ]:
pos = EarthPosition(x=0, y=0, height=1000)
psi = 0.5  # rad
TAS = 45  # m/s
controls0 = {'delta_elevator': 0, 'delta_aileron': 0, 'delta_rudder': 0, 'delta_t': 0.5}

In [ ]:
trimmed_state, trimmed_controls = steady_state_trim(
    aircraft,
    environment,
    pos,
    psi,
    TAS,
    controls0
)    

In [ ]:
trimmed_state

In [ ]:
trimmed_controls

Now, all the necessary elements in order to calculate forces and moments are available 

In [ ]:
# Environment conditions for the current state:
environment.update(trimmed_state)

# Forces and moments calculation:
forces, moments = aircraft.calculate_forces_and_moments(trimmed_state, environment, controls0)

In [ ]:
forces, moments

The aircraft is trimmed indeed: the total forces and moments (aerodynamics + gravity + thrust) are zero

## Simulation 

In order to simulate the dynamics of the aircraft under certain inputs in an environment, the user can set up a simulation using a dynamic system:

In [ ]:
from pyfme.models import EulerFlatEarth

In [ ]:
system = EulerFlatEarth(t0=0, full_state=trimmed_state)

### Constant Controls 

Let's set the controls for the aircraft during the simulation. As a first step we will set them constant and equal to the trimmed values.

In [ ]:
from pyfme.utils.input_generator import Constant

In [ ]:
controls = controls = {
    'delta_elevator': Constant(trimmed_controls['delta_elevator']),
    'delta_aileron': Constant(trimmed_controls['delta_aileron']),
    'delta_rudder': Constant(trimmed_controls['delta_rudder']),
    'delta_t': Constant(trimmed_controls['delta_t'])
}

In [ ]:
from pyfme.simulator import Simulation

In [ ]:
sim = Simulation(aircraft, system, environment, controls)

Once the simulation is set, the propagation can be performed:

In [ ]:
results = sim.propagate(10)

The results are returned in a DataFrame:

In [ ]:
results

In [ ]:
%matplotlib inline

In [ ]:
kwargs = {'marker': '.',
          'subplots': True,
          'sharex': True,
          'figsize': (12, 6)}

In [ ]:
results.plot(y=['x_earth', 'y_earth', 'height'], **kwargs);

In [ ]:
results.plot(y=['psi', 'theta', 'phi'], **kwargs);

In [ ]:
results.plot(y=['v_north', 'v_east', 'v_down'], **kwargs);

In [ ]:
results.plot(y=['p', 'q', 'r'], **kwargs);

In [ ]:
results.plot(y=['alpha', 'beta', 'TAS'], **kwargs);

In [ ]:
results.plot(y=['Fx', 'Fy', 'Fz'], **kwargs);

In [ ]:
results.plot(y=['Mx', 'My', 'Mz'], **kwargs);

In [ ]:
results.plot(y=['elevator', 'aileron', 'rudder', 'thrust'], **kwargs);

### Doublet 

Let's set the controls for the aircraft during the simulation. As a first step we will set them constant and equal to the trimmed values.

In [ ]:
from pyfme.utils.input_generator import Doublet

In [ ]:
de0 = trimmed_controls['delta_elevator']

In [ ]:
controls = controls = {
    'delta_elevator': Doublet(t_init=2, T=1, A=0.1, offset=de0),
    'delta_aileron': Constant(trimmed_controls['delta_aileron']),
    'delta_rudder': Constant(trimmed_controls['delta_rudder']),
    'delta_t': Constant(trimmed_controls['delta_t'])
}

In [ ]:
sim = Simulation(aircraft, system, environment, controls)

Once the simulation is set, the propagation can be performed:

In [ ]:
results = sim.propagate(90)

In [ ]:
results.plot(y=['x_earth', 'y_earth', 'height'], **kwargs);

In [ ]:
results.plot(y=['psi', 'theta', 'phi'], **kwargs);

In [ ]:
results.plot(y=['v_north', 'v_east', 'v_down'], **kwargs);

In [ ]:
results.plot(y=['p', 'q', 'r'], **kwargs);

In [ ]:
results.plot(y=['alpha', 'beta', 'TAS'], **kwargs);

In [ ]:
results.plot(y=['Fx', 'Fy', 'Fz'], **kwargs);

In [ ]:
results.plot(y=['Mx', 'My', 'Mz'], **kwargs);

In [ ]:
results.plot(y=['elevator', 'aileron', 'rudder', 'thrust'], **kwargs);

## Propagating only one time step

In [ ]:
dt = 0.05  # seconds
sim = Simulation(aircraft, system, environment, controls, dt)

In [ ]:
results = sim.propagate(0.5)
results

We can propagate for one time step even once the simulation has been propagated before:

In [ ]:
results = sim.propagate(sim.time+dt)
results

Notice that `results` will include the previous timesteps as well as the last one. To get just the last one one can use pandas `loc` or `iloc`:

In [ ]:
results.iloc[-1]  # last time step

In [ ]:
results.loc[sim.time]  # results for current simulation time